## pytorch hello world
* https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

In [52]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

In [55]:
x.shape, y.shape, x[0].shape

(torch.Size([64, 1000]), torch.Size([64, 10]), torch.Size([1000]))

In [56]:
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

99 31.906471252441406
199 0.2842945158481598
299 0.002175460569560528
399 7.674582775507588e-06
499 8.774386550669533e-09


## complex type tensor

- not supported yet : https://github.com/pytorch/pytorch/issues/755

In [57]:
import torch as th
import numpy as np

a = np.array([1+1j,2+2j])
b = np.array([3+3j,4+4j])
ath = th.from_numpy(a)
bth = th.from_numpy(b)
ath_cuda = ath.cuda()
ath_cuda += bth.cuda()
ath = ath_cuda.cpu()
print(ath.numpy())

TypeError: can't convert np.ndarray of type numpy.complex128. The only supported types are: double, float, float16, int64, int32, and uint8.

## imitate rx simulator

In [58]:
import numpy as np
import torch.nn as nn

In [59]:
def gen_rand_complex(low, high):
    r = np.random.uniform(low, high)
    i = np.random.uniform(low, high)
    return r + 1j * i

# UNIT TEST
t_value = gen_rand_complex(-1, 1)
print(t_value)
assert type(t_value) is complex, "error, type should be complex"

(-0.7443105126095386+0.0031990149596619055j)


simple linear model

In [73]:
class Model(nn.Module):
    def __init__(self, in_size, out_size):
        super(Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_size, 64),
            nn.ReLU(),
            nn.Linear(64, out_size),
            nn.ReLU()
        )
        self.learning_rate = 1e-4
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
    def forward(self, x):
        return self.model(x)
    
m = Model(2,2)
print(m)

loss_fn = nn.MSELoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)


Model(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=2, bias=True)
    (3): ReLU()
  )
)


In [74]:
tx_buf = []
rx_buf = []
train_batch_size = 10
iteration = 100
for i in range(iteration):
    tx = gen_rand_complex(-1, 1)
    rx = gen_rand_complex(-1, 1)
    if ((0 < i) and (i % train_batch_size == 0)) or (i == iteration - 1):
        # train
        x = torch.tensor([[x.real, x.imag] for x in rx_buf])
        y = torch.tensor([[x.real, x.imag] for x in tx_buf])
        y_pred = m.forward(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("train, loss : {}".format(loss))
        tx_buf = []
        rx_buf = []
    else:
        # save on buf
        tx_buf.append(tx)
        rx_buf.append(rx)
        print(".", end="")

..........train, loss : 0.4526260495185852
.........train, loss : 0.38767004013061523
.........train, loss : 0.44828706979751587
.........train, loss : 0.5068626403808594
.........train, loss : 0.40768566727638245
.........train, loss : 0.2955981492996216
.........train, loss : 0.5194939374923706
.........train, loss : 0.3902595639228821
.........train, loss : 0.6379255056381226
........train, loss : 0.23576787114143372


lstm model approach

In [ ]:
class ModelLstm(nn.Module):
    def __init__(self, in_size, out_size):
        super(ModelLstm, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(in_size, 64),
            nn.ReLU(),
            nn.Linear(64, out_size),
            nn.ReLU()
        )
        self.learning_rate = 1e-4
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
    def forward(self, x):
        return self.model(x)
    
m = Model(2,2)
print(m)

loss_fn = nn.MSELoss()
learning_rate = 1e-4
optimizer = torch.optim.Adam(m.parameters(), lr=learning_rate)

In [80]:
torch.randn(1, 1, 3)

tensor([[[-2.2627,  0.8289,  0.5312]]])

In [85]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

In [86]:
inputs

[tensor([[1.1286, 0.2937, 0.8111]]),
 tensor([[-0.0121, -0.3044, -0.5591]]),
 tensor([[-0.6684,  0.4180, -0.2707]]),
 tensor([[ 0.1936, -1.6071, -0.5119]]),
 tensor([[-0.1908,  0.5175,  1.9192]])]

In [87]:
torch.cat(inputs)

tensor([[ 1.1286,  0.2937,  0.8111],
        [-0.0121, -0.3044, -0.5591],
        [-0.6684,  0.4180, -0.2707],
        [ 0.1936, -1.6071, -0.5119],
        [-0.1908,  0.5175,  1.9192]])

In [88]:
torch.cat(inputs).view(len(inputs), 1, -1)

tensor([[[ 1.1286,  0.2937,  0.8111]],

        [[-0.0121, -0.3044, -0.5591]],

        [[-0.6684,  0.4180, -0.2707]],

        [[ 0.1936, -1.6071, -0.5119]],

        [[-0.1908,  0.5175,  1.9192]]])

In [84]:
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[ 0.2215,  0.2781,  0.1470]],

        [[ 0.0348,  0.2484,  0.0457]],

        [[-0.0827,  0.0845,  0.0975]],

        [[-0.0141, -0.0733,  0.0782]],

        [[-0.1010, -0.1885,  0.0506]]], grad_fn=<StackBackward>)
(tensor([[[-0.1010, -0.1885,  0.0506]]], grad_fn=<StackBackward>), tensor([[[-0.1664, -0.2707,  0.1635]]], grad_fn=<StackBackward>))
